In [0]:
import requests
import time
import json 

# --- Configurações ---
# Garanta que os valores no seu secret scope "meu-escopo" estão 100% corretos
DATABRICKS_HOST = 
DATABRICKS_TOKEN = 
SPACE_ID = 
BOT_TOKEN = dbutils.secrets.get(scope="meu-escopo", key="bot_token")

API_URL = f"https://api.telegram.org/bot{BOT_TOKEN}"

# --- Estado da conversa (será resetado a cada execução da célula) ---
conversation_id = None

# --- Funções ---
def get_updates(offset=None):
    url = f"{API_URL}/getUpdates"
    params = {"timeout": 30, "offset": offset}
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Erro ao buscar updates do Telegram: {e}")
        return {}

def send_message(chat_id, text):
    url = f"{API_URL}/sendMessage"
    payload = {"chat_id": str(chat_id), "text": str(text)}
    try:
        requests.post(url, data=payload)
    except requests.exceptions.RequestException as e:
        print(f"Erro ao enviar mensagem para o Telegram: {e}")

def ask_genie(question):
    global conversation_id

    headers = {
        "Authorization": f"Bearer {DATABRICKS_TOKEN}",
        "Content-Type": "application/json"
    }
    
    # Lógica para iniciar ou continuar a conversa
    if conversation_id is None:
        endpoint = f"/api/2.0/genie/spaces/{SPACE_ID}/start-conversation"
        payload = {
            "title": "Conversa Telegram",
            "messages": [{"role": "user", "content": question}]
        }
    else:
        endpoint = f"/api/2.0/genie/spaces/{SPACE_ID}/conversations/{conversation_id}/messages"
        payload = {
            "messages": [{"role": "user", "content": question}]
        }

    # Montagem final da URL e envio
    full_url = f"https://dbc-bb139269-194a.cloud.databricks.com{endpoint}"
    
    print(f"--- Disparando requisição para Genie ---")
    print(f"URL: {full_url}")
    print(f"PAYLOAD: {payload}")

    try:
        response = requests.post(full_url, headers=headers, data=json.dumps(payload))
        print(f"STATUS CODE: {response.status_code}")
        print(f"RESPONSE TEXT: {response.text}")
        print("---------------------------------------")

        if response.status_code == 200:
            data = response.json()
            if "conversation_id" in data:
                conversation_id = data["conversation_id"]
            return data["message"]["content"]
        else:
            return f"Erro na API Genie: {response.text}"
            
    except requests.exceptions.RequestException as e:
        print(f"Erro de conexão com a API Genie: {e}")
        return f"Erro de conexão com a API Genie: {e}"

# --- Loop principal ---
def main():
    offset = None
    print("Bot iniciado. Aguardando mensagens...")
    while True:
        try:
            updates = get_updates(offset)
            if "result" in updates:
                for update in updates.get("result", []):
                    offset = update["update_id"] + 1
                    message = update.get("message", {}).get("text")
                    chat_id = update.get("message", {}).get("chat", {}).get("id")

                    if message and chat_id:
                        print(f"Recebida mensagem: '{message}' do chat_id: {chat_id}")
                        resposta = ask_genie(message)
                        send_message(chat_id, resposta)
            
            time.sleep(1)
        except Exception as e:
            print(f"Um erro ocorreu no loop principal: {e}")
            time.sleep(5) # Evita spam de logs em caso de erro contínuo

# --- Ponto de entrada ---
if __name__ == "__main__":
    main()